# Protocol for Bone Analysis Workflow

## 0. Import Libraries and Parameters Setting

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from matplotlib.colors import Normalize
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
from scipy.spatial.distance import cdist
import math
import pickle
from scipy.spatial import KDTree
from scipy.optimize import minimize
import seaborn as sns
from scipy.spatial.distance import jensenshannon
from scipy.stats import entropy
from PIL import Image
from imaris_ims_file_reader.ims import ims
from skimage import measure, morphology
from collections import defaultdict
import re
from scipy.spatial import ConvexHull
from matplotlib.path import Path
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import Rbf
from matplotlib.patches import Patch
from scipy.interpolate import griddata
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score
import matplotlib.patches as mpatches

In [ ]:
os.getcwd()
os.chdir('/Users/fm07-admin/Projects/DisVis/')
# Increase the maximum image size limit
Image.MAX_IMAGE_PIXELS = None
data_dir_5fu = 'data_metabone_5fu'
save_dir = '/'

mk_color_map = {
    'MKs': "#FF00FF",  # Magenta 
    'Adipos': "#C96500",  # Brown 
    'Sinusoids': "#00FFFF",  # Cyan # Marker CD105 = Sinusoids
    'MSCs': '#FFCC00',  # Yellow # Marker Cxcl12 = MSCs
    'Neurons': "#FF7F00",  # Orange  TODO: the value is missing
    'Arteries': "#33A02C", # Green
    'GFP': "#A6CEE3" # Light Blue
}


hscs_color_map = {
    'HSCs': "#A6CEE3" ,  # Light Blue
    'RDs' : "#A9A9A9" # Gray
}

## 1. Data Import



### 1.1 Load CSV Files

In [ ]:


# Define a function to filter and get the Position.csv files by condition
def get_position_files_by_conditions():
    # Lists to store Position.csv files for each condition
    position_files_d5 = []
    position_files_d10 = []
    position_files_d15 = []
    position_files_d30 = []
    position_files_ss = []
    
    # Find all directories in data_dir_5fu, excluding .ims files
    all_dirs = [os.path.join(data_dir_5fu, d) for d in os.listdir(data_dir_5fu) if d.startswith('d')] # get all the directories starting with 2
    
    # Classify the directories into d5, d10, d15, d30, or "ss"
    for data_dir in all_dirs:
        
        sub_dirs = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('Statistics')]
        for sub_dir in sub_dirs:
            # Get the first file ending with 'Position.csv' in each sub-directory
            # First check for Position_Reference_Frame.csv, if not found, fallback to Position.csv
            position_file = [os.path.join(sub_dir, f) for f in os.listdir(sub_dir) if f.endswith('Position_Reference_Frame.csv')]
            if not position_file:
                # Only look for Position.csv if Position_Reference_Frame.csv is not found
                position_file = [os.path.join(sub_dir, f) for f in os.listdir(sub_dir) if f.endswith('Position.csv')]
        
            if position_file:
                if 'd5' in data_dir:
                    position_files_d5.append(position_file[0])
                elif 'd10' in data_dir:
                    position_files_d10.append(position_file[0])
                elif 'd15' in data_dir:
                    position_files_d15.append(position_file[0])
                elif 'd30' in data_dir:
                    position_files_d30.append(position_file[0])
                else:
                    # Files that don't contain 'd5', 'd10', 'd15', or 'd30' are classified as "ss"
                    position_files_ss.append(position_file[0])

    # Return a dictionary with the results
    return {
        'd5': position_files_d5,
        'd10': position_files_d10,
        'd15': position_files_d15,
        'd30': position_files_d30,
        'ss': position_files_ss
    }

# Get position files for d5, d10, d15, d30, and the "ss" case
position_files_by_conditions = get_position_files_by_conditions()
# position_files_by_conditions_hsc = get_position_files_by_conditions_hsc()

# Access individual lists for each condition
position_dirs_d5 = position_files_by_conditions['d5']
position_dirs_d10 = position_files_by_conditions['d10']
position_dirs_d15 = position_files_by_conditions['d15']
position_dirs_d30 = position_files_by_conditions['d30'] 
position_dirs_ss = position_files_by_conditions['ss']


In [ ]:
# Read the position files and create the DataFrames with columns 'Position.x', 'Position.y', 'Position.z'
# Regular expression to capture the word that starts with 'm' and extract the part after 'm'
pattern = r'\bm([A-Za-z0-9]+)'

def process_position_dirs(position_dirs, scale_factors):
    """
    Process a list of CSV files containing position data, scales the coordinates,
    and stores the resulting DataFrame in the provided output dictionary.
    
    Parameters:
    position_dirs (list): List of file paths to CSV files containing the position data.
    scale_factors (dict): Dictionary with scaling factors for 'x', 'y', and 'z'.
                        e.g., {'x': 0.7575, 'y': 0.7575, 'z': 2.5}
    Returns:
    None: The function modifies the provided output_dict in place.
    """
    output_dict = {}
    for position_dir in position_dirs:
        # print(position_dir.split('/')[-1])
        df = pd.read_csv(position_dir, skiprows=3)
        
        # Take the first 3 columns as the position coordinates
        df = df.iloc[:, :3]
        df.columns = ['Position.X', 'Position.Y', 'Position.Z']
        
        # Scale the position coordinates
        df['Position.X'] = df['Position.X'] / scale_factors['x']
        df['Position.Y'] = df['Position.Y'] / scale_factors['y']
        df['Position.Z'] = df['Position.Z'] / scale_factors['z']
        
        # Extract a key name for the output dictionary from the file path
        match = re.search(pattern, position_dir)
        if match:
            df_name_prefix = '_'.join(position_dir.split('/')[-1].split(' ')[:2])
            if match.group(1) == 'MK':
                df_name = '_'.join([df_name_prefix, 'MKs'])
            elif match.group(1) == 'Adipo':
                df_name = '_'.join([df_name_prefix, 'Adipos'])
            elif match.group(1) == 'Cxcl12':
                df_name = '_'.join([df_name_prefix, 'MSCs'])
            elif match.group(1) == 'CD105':
                df_name = '_'.join([df_name_prefix, 'Sinusoids'])
            else:
                df_name = '_'.join([df_name_prefix, match.group(1)])
            print(df_name)
            
        elif 'HSC' in position_dir:
            df_name_prefix = '_'.join(position_dir.split('/')[-1].split(' ')[:2])
            df_name = '_'.join([df_name_prefix, 'HSCs'])
            print(df_name)
            # df_name = position_dir.split('/')[-1].split('.')[0]
        elif 'RD' in position_dir:
            df_name_prefix = '_'.join(position_dir.split('/')[-1].split(' ')[:2])
            df_name = '_'.join([df_name_prefix, 'RDs'])
            print(df_name)
        elif 'Sca1' in position_dir:
            df_name_prefix = '_'.join(position_dir.split('/')[-1].split(' ')[:2])
            df_name = '_'.join([df_name_prefix, 'Arteries'])
            print(df_name)
        elif 'GFAP' in position_dir:
            df_name_prefix = '_'.join(position_dir.split('/')[-1].split(' ')[:2])
            df_name = '_'.join([df_name_prefix, 'Neurons'])
            print(df_name)
        
        # Store the processed DataFrame in the output dictionary
        output_dict[df_name] = df
        
        # Print summary statistics of the DataFrame
        # print(df.describe())
    return output_dict


# Define the scale factors
scale_factors = {'x': 0.7575, 'y': 0.7575, 'z': 2.5}

# Call the function for position_dirs_200901KK_st1 and position_dirs_201002KK_st3
positions_ss = process_position_dirs(position_dirs_ss, scale_factors)
positions_d5 = process_position_dirs(position_dirs_d5, scale_factors)
positions_d10 = process_position_dirs(position_dirs_d10, scale_factors)
positions_d15 = process_position_dirs(position_dirs_d15, scale_factors)
positions_d30 = process_position_dirs(position_dirs_d30, scale_factors)


### 1.2 Load Image Files (ims)


In [ ]:
# Get the GFP images of the bones
def get_ims_files_by_conditions():
    # Lists to store .ims files for each condition
    ims_files_d5 = []
    ims_files_d10 = []
    ims_files_d15 = []
    ims_files_d30 = []
    ims_files_ss = []
    
    # Find all directories in data_dir_5fu, excluding .ims files
    all_dirs = [os.path.join(data_dir_5fu, d) for d in os.listdir(data_dir_5fu) if d.startswith('d')] 
    
    # Classify the directories into d5, d10, d15, d30, or "ss"
    for data_dir in all_dirs:
        sub_dirs = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('ims') and 'GFP' in f]
        if 'd5' in data_dir:
            ims_files_d5.extend(sub_dirs)
        elif 'd10' in data_dir:
            ims_files_d10.extend(sub_dirs)
        elif 'd15' in data_dir:
            ims_files_d15.extend(sub_dirs)
        elif 'd30' in data_dir:
            ims_files_d30.extend(sub_dirs)
        else:
            # Files that don't contain 'd5', 'd10', 'd15', or 'd30' are classified as "ss"
            ims_files_ss.extend(sub_dirs)

    # Return a dictionary with the results
    return {
        'd5': ims_files_d5,
        'd10': ims_files_d10,
        'd15': ims_files_d15,
        'd30': ims_files_d30,
        'ss': ims_files_ss
    }


# Filter the ims files, keep the one with shorter name
def filter_ims_files(ims_list):
    # Dictionary to hold the shortest file for each unique prefix
    grouped_files = defaultdict(list)

    # Group filenames by the unique prefix after `d0/`
    for filepath in ims_list:
        # Extract the unique prefix (first two elements after 'd0/')
        parts = filepath.split('/')
        if len(parts) >= 3:
            unique_prefix = parts[2].split()[0] + ' ' + parts[2].split()[1]  # Example: '200901KK st1'
            grouped_files[unique_prefix].append(filepath)

    # Filter to keep only the shortest name in each group
    filtered_bone_ims = []
    for prefix, files in grouped_files.items():
        # Sort by filename length and take the shortest
        shortest_file = min(files, key=len)
        filtered_bone_ims.append(shortest_file)
    return filtered_bone_ims
        



# Convert the bone images with non-zero values to dataframes with Position.X and Position.Y columns
def image_to_df(image):
    # Find the non-zero intensities and their coordinates
    non_zero_coords = np.nonzero(image)  # Returns the y, x coordinates where intensity > 0
    # intensity_values = image[non_zero_coords]  # Extract the intensity values

    # Create a DataFrame from the non-zero coordinates and intensity values
    df = pd.DataFrame({
        'source': 'GFP',
        'Position.Z': 0, #non_zero_coords[0],
        'Position.Y': non_zero_coords[0],
        'Position.X': non_zero_coords[1]
    })
    
    return df




# Function to process and store images for a given condition
def process_bone_images(bone_ims_files):
    bone_dict = {}
    
    for i in bone_ims_files:
        # Extract bone name from the file path
        bone_name = '_'.join(i.split('/')[-1].split(' ')[:2])
        
        # Load and process the image (assuming ims function is defined elsewhere)
        bone_img = ims(i)  # You need to ensure that ims(i) correctly loads the image
        # bone_img = bone_img[0, bone_img.shape[1] - 1, :, :, :].max(axis=0).copy()
        bone_img = bone_img[0, 0, :, :, :].max(axis=0).copy()
        # Morphological operations, labeling and remove small objects
        # Threshold the image to create a binary mask
        binary_img = bone_img > 0
        # Label the binary image
        labeled_img = measure.label(binary_img)
        # Remove small objects
        # TODO: we need to try out the min_size
        cleaned_img = morphology.remove_small_objects(labeled_img, min_size=10000)
        # Convert the cleaned image back to binary
        bone_img_df = image_to_df(cleaned_img)
        # Store the processed image in the dictionary with the extracted name
        bone_dict[bone_name] = bone_img_df

    return bone_dict




In [ ]:
# Access individual lists for each condition
ims_files_by_conditions = get_ims_files_by_conditions()
bone_ss_ims = ims_files_by_conditions['ss']
bone_d5_ims = ims_files_by_conditions['d5']
bone_d10_ims = ims_files_by_conditions['d10']
bone_d15_ims = ims_files_by_conditions['d15']
bone_d30_ims = ims_files_by_conditions['d30']

# Filter the ims files, keep the one with shorter name
filtered_bone_ss_ims = filter_ims_files(bone_ss_ims)
filtered_bone_d5_ims = filter_ims_files(bone_d5_ims)
filtered_bone_d10_ims = filter_ims_files(bone_d10_ims)
filtered_bone_d15_ims = filter_ims_files(bone_d15_ims)
filtered_bone_d30_ims = filter_ims_files(bone_d30_ims)

# Process images for each condition
bone_ss = process_bone_images(filtered_bone_ss_ims)
bone_d5 = process_bone_images(filtered_bone_d5_ims)
bone_d10 = process_bone_images(filtered_bone_d10_ims)
bone_d15 = process_bone_images(filtered_bone_d15_ims)
bone_d30 = process_bone_images(filtered_bone_d30_ims)

### 1.3 Convert Image to Dataframe and Combine both Dataframes

In [ ]:
# From the positions_ss, positions_d5, positions_d10, positions_d15, positions_d30
def positions_concat(positions_dict, bone_dict):
    positions_df_dict = dict()
    keys = {}
    
    # Create a mapping of dataset and sources
    for key in positions_dict.keys():
        dataset = '_'.join(key.split('_')[:2])
        source = key.split('_')[-1]
        if dataset not in keys:
            keys[dataset] = [source]
        else:
            keys[dataset].append(source)
    
    # Concatenate DataFrames, adding a 'source' column
    for dataset, sources in keys.items():
        df = pd.concat(
            [positions_dict[f'{dataset}_{source}'].assign(source=source) for source in sources], 
            ignore_index=True
        )
        df = pd.concat([df, bone_dict[dataset]], ignore_index=True)
        positions_df_dict[dataset] = df
    return positions_df_dict

positions_ss_dfs = positions_concat(positions_ss, bone_ss)
positions_d5_dfs = positions_concat(positions_d5, bone_d5)
positions_d10_dfs = positions_concat(positions_d10, bone_d10)
positions_d15_dfs = positions_concat(positions_d15, bone_d15)
positions_d30_dfs = positions_concat(positions_d30, bone_d30)

In [ ]:
# delete bone_ss, bone_d5, bone_d10, bone_d15, bone_d30 to free up memory
del bone_ss, bone_d5, bone_d10, bone_d15, bone_d30

### 1.4 Load Reference Bone

In [ ]:
reference_bone_dir = 'data_metabone_5fu/reference_bone/200117KK st4 acat27a 5FU d5 mBone(1)  mGFP(2).ims'
reference_bone = ims(reference_bone_dir)

# ref_bone_outer = reference_bone[0,0,:,:,:].max(axis=0).copy()
ref_bone_inner = reference_bone[0,1,:,:,:].max(axis=0).copy()

# ref_bone_outer = image_to_df(ref_bone_outer)
ref_bone_inner = image_to_df(ref_bone_inner)

# Flip the y-axis to match the image coordinates
y_max = np.max((ref_bone_outer['Position.Y'].max(), ref_bone_inner['Position.Y'].max()))
# ref_bone_outer['Position.Y'] = y_max - ref_bone_outer['Position.Y']
ref_bone_inner['Position.Y'] = y_max - ref_bone_inner['Position.Y']

# delete reference bone to save memory
del reference_bone

## 2. Data Inspection and Adjustments

### 2.1 Visual Inspection

In [ ]:
# Visualize the x, y for each df 
for test_df in [positions_ss_dfs, positions_d5_dfs, positions_d10_dfs, positions_d15_dfs, positions_d30_dfs]:
    fig, axs = plt.subplots(len(test_df.keys()), 7, figsize=(12*len(test_df.keys()), 36))

    # Visualize the steady state positions row by key and the columns are source
    for i, key in enumerate(test_df.keys()):
        for j, source in enumerate(test_df[key]['source'].unique()):
            if source == 'GFP':
                sample_number = 10000
                source_df = test_df[key].loc[test_df[key]['source'] == source, ['Position.X', 'Position.Y']].sample(sample_number)
            else:
                source_df = test_df[key].loc[test_df[key]['source'] == source, ['Position.X', 'Position.Y']]
            ax = axs[i, j]
            ax.scatter(source_df['Position.X'], source_df['Position.Y'], s=1)
            ax.set_title(f"{key} - {source}")
            ax.axis('equal')
            ax.set_xlabel('Position.X')
            ax.set_ylabel('Position.Y')
            
    plt.tight_layout()
    # fig.savefig(f'{save_dir}/{datetime.today().strftime("%y%m%d")}_YL_raw_d5_d10_d15_d30.png')
    plt.show()


### 2.2 Data Adjustments (Flipping Coordinates, etc.)

In [ ]:
# Based on the visual check above, we need to flip the y axis for dataset with key 
# ss: '201002KK_st5'
# d5: '200117KK_st3', 
# d10: '200110KK_st3', 
# d15: '200908KK_st4', '240819JH_st2'
# The flip should be applied to all sources but the shift should only based on the MK source
def y_flip(positions_df_dict, keys_to_flip):
    for key in keys_to_flip:
        # y_min = positions_df_dict[key]['Position.Y'].min()
        y_max = positions_df_dict[key]['Position.Y'].max()
        # Flip the y axis, we don't care about the source
        positions_df_dict[key]['Position.Y'] = y_max - positions_df_dict[key]['Position.Y']
    return positions_df_dict

positions_ss_dfs = y_flip(positions_ss_dfs, ['201002KK_st5', '200901KK_st3', '210518KK_st12', '210518KK_st1', '200728KK_st2'])
positions_d5_dfs = y_flip(positions_d5_dfs, ['200117KK_st3', '200117KK_st5', '200117KK_st4','200117KK_st1'])
positions_d10_dfs = y_flip(positions_d10_dfs, ['200110KK_st3','200110KK_st1', '191202KK_st1', '200110KK_st4', '200110KK_st8'])
positions_d15_dfs = y_flip(positions_d15_dfs, ['240819JH_st2', '201002KK_st7', '201020KK_st1', '200901KK_st5', '200728KK_st3', '200908KK_st4', '210108KK_st10'])
positions_d30_dfs = y_flip(positions_d30_dfs, ['200901KK_st7', '201103KKI_st4', '201103KK_st2', '200908KK_st2', '240926JH_st2'])


def x_flip(positions_df_dict, keys_to_flip):
    for key in keys_to_flip:
        # x_min = positions_df_dict[key]['Position.X'].min()
        x_max = positions_df_dict[key]['Position.X'].max()
        # Flip the x axis, we don't care about the source
        positions_df_dict[key]['Position.X'] = x_max - positions_df_dict[key]['Position.X']
    return positions_df_dict

positions_d5_dfs = x_flip(positions_d5_dfs, ['200117KK_st5'])


In [ ]:
# Visualize the x, y for each df again after flipping the coordinates
for test_df in [positions_ss_dfs, positions_d5_dfs, positions_d10_dfs, positions_d15_dfs, positions_d30_dfs]:
    fig, axs = plt.subplots(len(test_df.keys()), 7, figsize=(12*len(test_df.keys()), 36))

    # Visualize the steady state positions row by key and the columns are source
    for i, key in enumerate(test_df.keys()):
        for j, source in enumerate(test_df[key]['source'].unique()):
            if source == 'GFP':
                sample_number = 10000
                source_df = test_df[key].loc[test_df[key]['source'] == source, ['Position.X', 'Position.Y']].sample(sample_number)
            else:
                source_df = test_df[key].loc[test_df[key]['source'] == source, ['Position.X', 'Position.Y']]
            ax = axs[i, j]
            ax.scatter(source_df['Position.X'], source_df['Position.Y'], s=1)
            ax.set_title(f"{key} - {source}")
            ax.axis('equal')
            ax.set_xlabel('Position.X')
            ax.set_ylabel('Position.Y')
            
    plt.tight_layout()
    # fig.savefig(f'{save_dir}/{datetime.today().strftime("%y%m%d")}_YL_raw_d5_d10_d15_d30.png')
    plt.show()


## 3. Bone Alignment and Transformation

### 3.1 Align Bones to the Reference Bone

In [ ]:
# Find the 2D outline of the bone in the x-y plane
def find_outline(points, window_size=10):
    """
    Find the outline of a set of points by finding the min and max y-values for each x-value within a window.
    The outline is only in the x-y plane.
    Parameters:
    points : np.array of shape (n,2) points.
    window_size : size of the window to smooth the outline.
    Returns:
    outline_points : np.array of outline points.
    """
    df = pd.DataFrame(points, columns=['x', 'y'])
    
    min_y_points = []
    max_y_points = []

    # Sort points by x value
    df_sorted = df.sort_values(by='x')
    
    # Slide over the x values with a window
    for i in range(0, len(df_sorted), window_size):
        window = df_sorted.iloc[i:i + window_size]
        min_y = window.loc[window['y'].idxmin()]
        max_y = window.loc[window['y'].idxmax()]
        min_y_points.append(min_y)
        max_y_points.append(max_y)
    
    # Ensure the outline is in order
    min_y_points = pd.DataFrame(min_y_points).drop_duplicates().sort_values(by='x').values
    max_y_points = pd.DataFrame(max_y_points).drop_duplicates().sort_values(by='x', ascending=False).values

    # Combine min_y and max_y points and close the loop
    outline_points = np.vstack([min_y_points, max_y_points, min_y_points[0]])

    return outline_points

# 0. Calculate the overlap
# 1. Find the outline of each bone based on the x-y plane

# 2. Find the center of each bone (outline) and put the center of the bones at the same position
def calculate_centroid(outline_points):
    """
    Calculate the centroid of the bone outline.
    
    Parameters:
    outline_points : np.array of shape (n, 2)
    
    Returns:
    centroid : tuple containing (centroid_x, centroid_y)
    """
    # Use the weight centroid
    centroid_x = np.mean(outline_points[:, 0])
    centroid_y = np.mean(outline_points[:, 1])
    
    # Using the middle of the x and y values as the centroid
    # x_min, x_max = outline_points[:, 0].min(), outline_points[:, 0].max()
    # y_min, y_max = outline_points[:, 1].min(), outline_points[:, 1].max()
    # centroid_x = (x_min + x_max) / 2
    # centroid_y = (y_min + y_max) / 2
    return centroid_x, centroid_y
def translate_to_origin(outline_points, centroid):
    """
    Translate the outline points so that the centroid is at the origin.
    
    Parameters:
    outline_points : np.array of shape (n, 2)
    centroid : tuple containing (centroid_x, centroid_y)
    
    Returns:
    translated_points : np.array of shape (n, 2)
    """
    translated_points = outline_points.astype(np.float64).copy()
    
    translated_points[:, 0] -= centroid[0]
    translated_points[:, 1] -= centroid[1]
    return translated_points
# 3. Rescale the bones to the same size (bounding box)(optional)
def get_max_dimensions(bone_dicts):
    """
    Find the maximum width and height across all bone outlines in the given dictionaries.
    
    Parameters:
    bone_dicts : list of dictionaries of bones (where each value is a DataFrame with 'Position.X' and 'Position.Y')
    
    Returns:
    max_width : float, maximum width found across all bones
    max_height : float, maximum height found across all bones
    """
    max_width = 0
    max_height = 0
    
    for bone_dict in bone_dicts:
        for df in bone_dict.values():
            # Extract bone points where 'source' == 'Bone'
            bone_points = df[df['source'] == 'Bone'][['Position.X', 'Position.Y']].values
            
            # Find min and max values of x and y
            min_x, max_x = bone_points[:, 0].min(), bone_points[:, 0].max()
            min_y, max_y = bone_points[:, 1].min(), bone_points[:, 1].max()
            
            # Calculate width and height
            width = max_x - min_x
            height = max_y - min_y
            
            # Update maximum width and height if necessary
            if width > max_width:
                max_width = width
            if height > max_height:
                max_height = height
                
    return max_width, max_height
def rescale_outline(outline_points, max_width, max_height):
    """
    Rescale the bone outline to fit within the maximum width and height across all bones.
    
    Parameters:
    outline_points : np.array of shape (n, 2)
    max_width : float, the maximum width across all bones
    max_height : float, the maximum height across all bones
    
    Returns:
    scaled_points : np.array of shape (n, 2)
    """
    min_x, max_x = outline_points[:, 0].min(), outline_points[:, 0].max()
    min_y, max_y = outline_points[:, 1].min(), outline_points[:, 1].max()

    current_width = max_x - min_x
    current_height = max_y - min_y

    scale_x = max_width / current_width
    scale_y = max_height / current_height

    scaled_points = outline_points.copy()
    scaled_points[:, 0] *= scale_x
    scaled_points[:, 1] *= scale_y

    return scaled_points

# Calculate the overlap area on the grid inside the outline
def calculate_overlap_area(reference_outline, target_outline, resolution=200):
    """
    Calculate the overlap area (in terms of pixels or points) between two outlines.
    
    Parameters:
    reference_outline : np.array of shape (n, 2), outline of the reference bone
    target_outline : np.array of shape (n, 2), outline of the target bone
    resolution : int, the number of points or pixels to use for the area calculation.
    
    Returns:
    overlap_area : float, the number of pixels or points where the areas overlap.
    """
    # Ensure the outlines are 2D arrays of shape (N, 2)
    reference_outline = np.asarray(reference_outline).reshape(-1, 2)
    target_outline = np.asarray(target_outline).reshape(-1, 2)
    
    # Calculate centroids for both bones
    reference_centroid = calculate_centroid(reference_outline)
    target_centroid = calculate_centroid(target_outline)
    
    # Translate both bones to center them
    reference_outline_centered = translate_to_origin(reference_outline, reference_centroid)
    target_outline_centered = translate_to_origin(target_outline, target_centroid)
    
    # Get bounding box of the reference outline
    min_x, max_x = reference_outline_centered[:, 0].min(), reference_outline_centered[:, 0].max()
    min_y, max_y = reference_outline_centered[:, 1].min(), reference_outline_centered[:, 1].max()
    
    # Generate grid of points (pixels) covering the bounding box
    x_grid = np.linspace(min_x, max_x, resolution)
    y_grid = np.linspace(min_y, max_y, resolution)
    xv, yv = np.meshgrid(x_grid, y_grid)
    grid_points = np.vstack([xv.ravel(), yv.ravel()]).T

    # Create Path objects for the reference and target outlines
    reference_path = Path(reference_outline_centered)
    target_path = Path(target_outline_centered)
    
    # Check which points of the grid are inside both outlines
    points_in_reference = reference_path.contains_points(grid_points)
    points_in_target = target_path.contains_points(grid_points)
    
    # Calculate the overlap area as the number of points (pixels) inside both outlines
    overlap_area = np.sum(points_in_reference & points_in_target)
    
    return overlap_area

def rotate_points(points, angle):
    """
    Rotate a set of points by a given angle.
    
    Parameters:
    points : np.array of shape (n, 2)
    angle : float, angle to rotate by in radians
    
    Returns:
    rotated_points : np.array of shape (n, 2)
    """
    # Ensure points are a 2D array of shape (N, 2)
    points = np.asarray(points).reshape(-1, 2)
    
    # Rotation matrix
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], 
                                [np.sin(angle), np.cos(angle)]])
    
    # Rotate points
    rotated_points = points.dot(rotation_matrix)
    
    return rotated_points

# TODO: for some reason the function seems only test on the initial guesses, so now we are using the grid search
def optimize_rotation(reference_outline, target_outline):
    """
    Optimize the rotation of the target bone to maximize overlap with the reference bone,
    constrained to -90 to 90 degrees.
    
    Parameters:
    reference_outline : np.array of shape (n, 2), outline of the reference bone
    target_outline : np.array of shape (n, 2), outline of the target bone
    
    Returns:
    optimal_rotated_points : np.array of shape (n, 2), the rotated and recentered target bone outline with maximum overlap
    optimal_angle : float, the optimal rotation angle in radians
    """
    
    def objective_function(angle):
        # Rotate the target outline by the current angle
        rotated_outline = rotate_points(target_outline, angle)
        # Calculate the negative overlap (since we are minimizing)
        return -calculate_overlap_area(reference_outline, rotated_outline)

    # Try different initial guesses for the angle to improve convergence
    initial_guesses = [-0.1, 0, 0.1]  # You can try different values here
    
    best_result = None
    best_angle = None
    best_rotated_points = None

    for x0 in initial_guesses:
        # Perform optimization with the initial guess x0
        result = minimize(objective_function, x0=x0, bounds=[(-np.pi/4, np.pi/4)], method='L-BFGS-B', options={'maxiter': 50})
        
        # Check if this result is valid and better than the current best
        if result.success:
            if best_result is None or result.fun < best_result:
                best_result = result.fun
                best_angle = result.x[0]
                best_rotated_points = rotate_points(target_outline, best_angle)

    if best_rotated_points is None:
        raise ValueError("Optimization failed to converge.")

    # Recenter the final rotated outline
    final_centroid = calculate_centroid(best_rotated_points)
    best_rotated_points = translate_to_origin(best_rotated_points, final_centroid)
    
    return best_rotated_points, best_angle, final_centroid

def grid_search_rotation(reference_outline, target_outline, angle_step=np.pi/36):
    """
    Perform a grid search over possible rotation angles to maximize overlap.
    
    Parameters:
    reference_outline : np.array of shape (n, 2), outline of the reference bone
    target_outline : np.array of shape (n, 2), outline of the target bone
    angle_step : float, step size for angle search (in radians)
    
    Returns:
    best_rotated_points : np.array of shape (n, 2), the rotated target bone outline with maximum overlap
    best_angle : float, the optimal rotation angle in radians
    """
    best_angle = None
    max_overlap = -np.inf
    best_rotated_points = None
    
    # Iterate over angles between -90 and 90 degrees (in radians)
    for angle in np.arange(-np.pi/4, np.pi/4, angle_step):
        rotated_outline = rotate_points(target_outline, angle)
        overlap = calculate_overlap_area(reference_outline, rotated_outline)
        
        if overlap > max_overlap:
            max_overlap = overlap
            best_angle = angle
            best_rotated_points = rotated_outline
    
    # Recenter the final rotated outline
    final_centroid = calculate_centroid(best_rotated_points)
    best_rotated_points = translate_to_origin(best_rotated_points, final_centroid)
    
    return best_rotated_points, best_angle, final_centroid

# Modify the process_and_align_bones function to accept the max_width and max_height
def process_and_align_bones_with_overlap(bone_dict, reference_bone_name='201002KK_st5', window_size=500):
    """
    Process and align all bones from the given dictionary to maximize overlap with a reference bone.
    
    Parameters:
    bone_dict : dict of bones, where each value is a DataFrame with 'Position.X' and 'Position.Y'.
    reference_bone_name : string, the name of the bone to use as the reference for alignment.
    
    Returns:
    aligned_bones : dict of aligned bone outlines
    """
    aligned_bones = {}
    aligned_angles = {}
    aligned_centroids = {}
    # Check the type of the reference_bone_name
    # If it is a dataframe, we can use the reference_bone_name directly
    if isinstance(reference_bone_name, pd.DataFrame):
        reference_df = reference_bone_name
        reference_points = reference_df[reference_df['source'] == 'GFP'][['Position.X', 'Position.Y']].values
        reference_outline = find_outline(reference_points, window_size=window_size)
        reference_centroid = calculate_centroid(reference_outline)
        reference_outline = translate_to_origin(reference_outline, reference_centroid)
        
        for bone_name, df in bone_dict.items():

            # Filter points where 'source' == 'Bone'
            bone_points = df[df['source'] == 'GFP'][['Position.X', 'Position.Y']].values
        
            # Find the outline of the target bone
            target_outline = find_outline(bone_points, window_size=window_size)
            # Optimize rotation to maximize overlap with the reference bone
            aligned_outline, best_angle, final_centroid = grid_search_rotation(reference_outline, target_outline)
            aligned_bones[bone_name] = aligned_outline
            aligned_angles[bone_name] = best_angle
            aligned_centroids[bone_name] = final_centroid

        
    elif isinstance(reference_bone_name, str):
            
        
        # Extract the reference bone outline
        reference_df = positions_ss_dfs[reference_bone_name]
        reference_points = reference_df[reference_df['source'] == 'Bone'][['Position.X', 'Position.Y']].values

        reference_outline = find_outline(reference_points, window_size=window_size)
        # Recenter the reference outline
        reference_centroid = calculate_centroid(reference_outline)
        reference_outline = translate_to_origin(reference_outline, reference_centroid)
        
        for bone_name, df in bone_dict.items():

            if bone_name == reference_bone_name:
                # Keep the reference bone as is
                aligned_bones[bone_name] = reference_outline
                aligned_angles[bone_name] = 0
                aligned_centroids[bone_name] = reference_centroid
            else:
                # Filter points where 'source' == 'Bone'
                bone_points = df[df['source'] == 'Bone'][['Position.X', 'Position.Y']].values
            
                # Find the outline of the target bone
                target_outline = find_outline(bone_points, window_size=window_size)
                # Optimize rotation to maximize overlap with the reference bone
                aligned_outline, best_angle, final_centroid = grid_search_rotation(reference_outline, target_outline)
                aligned_bones[bone_name] = aligned_outline
                aligned_angles[bone_name] = best_angle
                aligned_centroids[bone_name] = final_centroid
    
    return aligned_bones, aligned_angles, aligned_centroids


In [ ]:
# Compare the outline found based on the ref_bone_outer and the ref_bone_inner
# recenter and find the outline
# ref_bone_outer_points = ref_bone_outer[ref_bone_outer['source'] == 'GFP'][['Position.X', 'Position.Y']].values
ref_bone_inner_points = ref_bone_inner[ref_bone_inner['source'] == 'GFP'][['Position.X', 'Position.Y']].values

# ef_bone_outer_outline = find_outline(ref_bone_outer_points, window_size=500)
ref_bone_inner_outline = find_outline(ref_bone_inner_points, window_size=500)
# ref_bone_outer_centroid = calculate_centroid(ref_bone_outer_outline)
ref_bone_inner_centroid = calculate_centroid(ref_bone_inner_outline)

# The two shared the same centroid
# ref_bone_outer_outline = translate_to_origin(ref_bone_outer_outline, ref_bone_outer_centroid)
# ref_bone_inner_outline = translate_to_origin(ref_bone_inner_outline, ref_bone_outer_centroid)
ref_bone_inner_outline = translate_to_origin(ref_bone_inner_outline, ref_bone_inner_centroid)

fig, ax = plt.subplots(figsize=(15, 5))
# ax.plot(ref_bone_outer_outline[:, 0], ref_bone_outer_outline[:, 1], color='r', label='Outer')
ax.plot(ref_bone_inner_outline[:, 0], ref_bone_inner_outline[:, 1], color='b', label='Inner')
ax.set_title('Outer and Inner Bone Outlines')

plt.legend()
plt.show()

In [ ]:

# Apply the process to all bone dictionaries using the max width and height
aligned_ss_bone_outlines, aligned_ss_angles, aligned_ss_centroids = process_and_align_bones_with_overlap(positions_ss_dfs, reference_bone_name=ref_bone_inner, window_size=500)
aligned_d5_bone_outlines, aligned_d5_angles, aligned_d5_centroids= process_and_align_bones_with_overlap(positions_d5_dfs, reference_bone_name=ref_bone_inner, window_size=500)
aligned_d10_bone_outlines, aligned_d10_angles, aligned_d10_centroids = process_and_align_bones_with_overlap(positions_d10_dfs, reference_bone_name=ref_bone_inner, window_size=500)
aligned_d15_bone_outlines, aligned_d15_angles, aligned_d15_centroids = process_and_align_bones_with_overlap(positions_d15_dfs, reference_bone_name=ref_bone_inner, window_size=500)
aligned_d30_bone_outlines, aligned_d30_angles, aligned_d30_centroids = process_and_align_bones_with_overlap(positions_d30_dfs, reference_bone_name=ref_bone_inner, window_size=500)

In [ ]:
# Applied the rotation and recenter to all the populations and save the aligned bones

def align_bones_with_centroids_angles(positions_df, aligned_centroids, aligned_angles):
    aligned_bones = {}
    
    for bone_name, df in positions_df.items():
        # Get the centroid of the bone
        centroid = aligned_centroids[bone_name]
        # Get the angle to rotate
        angle = aligned_angles[bone_name]

        # The order of the rotation and the translation matters!
        # The centroids are based on the rotated and translated points
        # Rotate the df
        df_rotated = df.copy()
        # Exclude the Bone points
        # TODO: if we still want to find the combined outline, then we will need to perform the rotation and recentering on the 'Bone' as well
        # df_rotated = df_rotated[df_rotated['source'] != 'Bone']
        df_rotated[['Position.X', 'Position.Y']] = rotate_points(df_rotated[['Position.X', 'Position.Y']].values, angle)
        
        # Translate the df
        df_translated = df_rotated.copy()
        df_translated['Position.X'] -= centroid[0]
        df_translated['Position.Y'] -= centroid[1]
        
        aligned_bones[bone_name] = df_translated
    return aligned_bones


aligned_ss_bones = align_bones_with_centroids_angles(positions_ss_dfs, aligned_ss_centroids, aligned_ss_angles)
aligned_d5_bones = align_bones_with_centroids_angles(positions_d5_dfs, aligned_d5_centroids, aligned_d5_angles)
aligned_d10_bones = align_bones_with_centroids_angles(positions_d10_dfs, aligned_d10_centroids, aligned_d10_angles)
aligned_d15_bones = align_bones_with_centroids_angles(positions_d15_dfs, aligned_d15_centroids, aligned_d15_angles)
aligned_d30_bones = align_bones_with_centroids_angles(positions_d30_dfs, aligned_d30_centroids, aligned_d30_angles)


In [ ]:
'''
only will be used if we want to compare the aligned data with transformed data
# Concat all the dataframe from aligned_bones
aligned_positions_ss_df = pd.concat(aligned_ss_bones)
aligned_positions_d5_df = pd.concat(aligned_d5_bones)
aligned_positions_d10_df = pd.concat(aligned_d10_bones)
aligned_positions_d15_df = pd.concat(aligned_d15_bones)
aligned_positions_d30_df = pd.concat(aligned_d30_bones)

# Before we save the file, we exclude the data when source is 'Bone'
aligned_positions_ss_df = aligned_positions_ss_df[aligned_positions_ss_df['source'] != 'GFP']
aligned_positions_d5_df = aligned_positions_d5_df[aligned_positions_d5_df['source'] != 'GFP']
aligned_positions_d10_df = aligned_positions_d10_df[aligned_positions_d10_df['source'] != 'GFP']
aligned_positions_d15_df = aligned_positions_d15_df[aligned_positions_d15_df['source'] != 'GFP']
aligned_positions_d30_df = aligned_positions_d30_df[aligned_positions_d30_df['source'] != 'GFP']

# Save the data points 
aligned_positions_ss_df.to_csv(f'{save_dir}/aligned_positions_ss_df.csv', index=False)
aligned_positions_d5_df.to_csv(f'{save_dir}/aligned_positions_d5_df.csv', index=False)
aligned_positions_d10_df.to_csv(f'{save_dir}/aligned_positions_d10_df.csv', index=False)
aligned_positions_d15_df.to_csv(f'{save_dir}/aligned_positions_d15_df.csv', index=False)
aligned_positions_d30_df.to_csv(f'{save_dir}/aligned_positions_d30_df.csv', index=False)
'''

In [ ]:
# Delete positions_ss_dfs, positions_d5_dfs, positions_d10_dfs, positions_d15_dfs, positions_d30_dfs to save memory
# These dataframes are important for the alignment, outline creation
# Of course, we can create the new outline based on the aligned data (dict)
# TODO: delete them after we confirm the outline for each bone
# del positions_ss_dfs, positions_d5_dfs, positions_d10_dfs, positions_d15_dfs, positions_d30_dfs

### 3.2 Transform Bone Coordinates

In [ ]:
# Smooth the outlines
def smooth_outline(outline, sigma=2):
    if not np.array_equal(outline[0], outline[-1]):
        outline = np.vstack([outline, outline[0]])
    smoothed_x = gaussian_filter1d(outline[:, 0], sigma=sigma)
    smoothed_y = gaussian_filter1d(outline[:, 1], sigma=sigma)
    smoothed_outline = np.vstack((smoothed_x, smoothed_y)).T
    if not np.array_equal(smoothed_outline[0], smoothed_outline[-1]):
        smoothed_outline = np.vstack([smoothed_outline, smoothed_outline[0]])
    return smoothed_outline


In [ ]:
# Smooth the reference bone outlines
ref_bone_inner_smoothed = smooth_outline(ref_bone_inner_outline, sigma=80)
common_outline = ref_bone_inner_smoothed

In [ ]:
# Smoothed the alinged_bone_outlines
aligned_ss_bone_outlines_smoothed = {}
aligned_d5_bone_outlines_smoothed = {}
aligned_d10_bone_outlines_smoothed = {}
aligned_d15_bone_outlines_smoothed = {}
aligned_d30_bone_outlines_smoothed = {}
# TODO: problem with the d15 bone outline (because of the discontinious tail)
sigma = 50
for bone_name, outline in aligned_ss_bone_outlines.items():
    smoothed_outline = smooth_outline(outline, sigma=sigma)
    aligned_ss_bone_outlines_smoothed[bone_name] = smoothed_outline
for bone_name, outline in aligned_d5_bone_outlines.items():
    smoothed_outline = smooth_outline(outline, sigma=sigma)
    aligned_d5_bone_outlines_smoothed[bone_name] = smoothed_outline
for bone_name, outline in aligned_d10_bone_outlines.items():
    smoothed_outline = smooth_outline(outline, sigma=sigma)
    aligned_d10_bone_outlines_smoothed[bone_name] = smoothed_outline
for bone_name, outline in aligned_d15_bone_outlines.items():
    smoothed_outline = smooth_outline(outline, sigma=sigma)
    aligned_d15_bone_outlines_smoothed[bone_name] = smoothed_outline
for bone_name, outline in aligned_d30_bone_outlines.items():
    smoothed_outline = smooth_outline(outline, sigma=sigma)
    aligned_d30_bone_outlines_smoothed[bone_name] = smoothed_outline


In [ ]:
def get_y_range_at_x(shape_points, x):
    """
    Find the range of y-values where the vertical line at x intersects the shape.
    """
    # Find all edges of the shape where x is between the x-coordinates of the endpoints
    y_vals = []
    for i in range(len(shape_points)):
        p1 = shape_points[i]
        p2 = shape_points[(i + 1) % len(shape_points)]  # wrap around the shape points
        
        # Check if the x value is between p1 and p2's x-coordinates
        if (p1[0] <= x <= p2[0]) or (p2[0] <= x <= p1[0]):
            # Linearly interpolate to find the corresponding y value at x
            if p1[0] != p2[0]:  # Avoid division by zero
                y = p1[1] + (p2[1] - p1[1]) * (x - p1[0]) / (p2[0] - p1[0])
                y_vals.append(y)
    
    if y_vals:
        return min(y_vals), max(y_vals)
    else:
        return None, None  # No intersection with the shape at this x

def create_structured_grid(shape_points, x_num, y_num):
    """
    Create a structured grid by dividing the bounding box of the shape into x_num vertical sections.
    Then place y_num points along each vertical grid line where it intersects the shape.
    """
    shape_points = np.array(shape_points)
    
    # Step 1: Compute the bounding box
    min_x, max_x = np.min(shape_points[:, 0]), np.max(shape_points[:, 0])
    
    # Step 2: Divide the x-range into equal sections
    x_vals = np.linspace(min_x, max_x, x_num + 1)  # x_num divisions create x_num + 1 grid lines
    # Shift the x_vals to the left by half the grid spacing to center the grid
    x_vals = x_vals[1:]  # Remove the first point (left edge of bounding box)
    x_vals = x_vals - (x_vals[1] - x_vals[0]) / 2  # Shift left by half the grid spacing
    
    grid_points = []

    # Step 3: For each x grid line, find the y range and then place points
    for x in x_vals:  # Skip the first and last lines (already have the bounding box)
        y_min, y_max = get_y_range_at_x(shape_points, x)
        
        if y_min is not None and y_max is not None:
            # Get y points by placing y_num points between y_min and y_max
            y_vals = np.linspace(y_min, y_max, y_num+1)
            
            # Shift the y_vals down by half the grid spacing to center the grid
            y_vals = y_vals[1:]  # Remove the first point (bottom edge of bounding box)
            y_vals = y_vals - (y_vals[1] - y_vals[0]) / 2  # Shift down by half the grid spacing
            # Add grid points (x, y) for this vertical line
            for y in y_vals: # Skip the first and last points (already have the y_min and y_max)
                grid_points.append([x, y])

    return np.array(grid_points)

def is_point_inside_shape(point, shape_points):
    """
    Determines if a point is inside an irregular shape using ray-casting.
    """
    x, y = point
    n = len(shape_points)
    inside = False
    p1x, p1y = shape_points[0]
    for i in range(n + 1):
        p2x, p2y = shape_points[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside


def thin_plate_spline_transform(src_points, dst_points):
    """
    Perform Thin Plate Spline (TPS) transformation from src_points to dst_points.
    """
    # Create Radial Basis Function (RBF) interpolators for x and y coordinates
    rbf_x = Rbf(src_points[:, 0], src_points[:, 1], dst_points[:, 0], function='thin_plate')
    rbf_y = Rbf(src_points[:, 0], src_points[:, 1], dst_points[:, 1], function='thin_plate')
    
    def transform(points):
        new_x = rbf_x(points[:, 0], points[:, 1])
        new_y = rbf_y(points[:, 0], points[:, 1])
        return np.vstack([new_x, new_y]).T
    
    return transform

def transform_data(data_points, grid_shape_1, grid_shape_2):
    """
    Apply the TPS transformation to the data points based on the grid transformation.
    """
    # Perform Thin Plate Spline (TPS) transformation
    tps_transform = thin_plate_spline_transform(grid_shape_2, grid_shape_1)
    
    # Apply the transformation to the data points
    transformed_data_points = tps_transform(data_points)
    
    return transformed_data_points


def transform_bone_positions(outline_dict, position_dict, common_outline, x_num=40, y_num=20, source_value=None):
    """
    Transforms the bone positions from multiple datasets using Thin Plate Spline (TPS) based on the provided outlines and positions.
    Parameters:
        outline_dict: Dictionary containing outlines.
        position_dict: Dictionary containing bone positions (DataFrames).
        common_outline: The common outline (to which the other outlines will be aligned).
        x_num: Number of vertical sections for structured grid.
        y_num: Number of horizontal points along each vertical section.
        source_value: If provided, filter the positions based on the specified source value.
    Returns:
        Dictionary containing the transformed bone positions with the 'source' column retained.
    """
    transformed_dict = {}

    # Create the structured grid for the common outline
    grid_common_outline = create_structured_grid(common_outline, x_num=x_num, y_num=y_num)

    # Loop through each dataset in the position_dict
    for dataset_name, position_df in position_dict.items():
        # Get the corresponding outline
        outline_2 = outline_dict[dataset_name]
        
        # Create the structured grid for the specific dataset's outline
        grid_outline_2 = create_structured_grid(outline_2, x_num=x_num, y_num=y_num)

        # Filter the positions based on the source (if provided)
        bone_positions_2, source_column = filter_bone_positions(position_df, source_value=source_value)

        # Transform the filtered bone positions from the dataset outline to the common outline
        transformed_bone_positions = transform_data(bone_positions_2, grid_common_outline, grid_outline_2)

        # Convert the transformed positions to a DataFrame and include the source column
        transformed_df = pd.DataFrame(transformed_bone_positions, columns=['Position.X', 'Position.Y'])
        transformed_df['source'] = source_column  # Add the source column back

        # Store the transformed DataFrame in the result dictionary
        transformed_dict[dataset_name] = transformed_df

    return transformed_dict


def filter_bone_positions(df, source_value=None):
    """
    Filters the DataFrame for a specific source if provided and returns the Position.X, Position.Y columns as a NumPy array.
    If source_value is None, return all positions.
    """
    if source_value:
        filtered_df = df[df['source'] == source_value]
    else:
        # Exclude the data with source value 'GFP'
        filtered_df = df[df['source'] != 'GFP']
    positions = filtered_df[['Position.X', 'Position.Y']].to_numpy()
    return positions, filtered_df['source'].to_numpy()  # Return positions and the source column



In [ ]:
x_num = 200
y_num = 35
# 10 mins with 200*35
# We are using the aligned_bone_date dict instead of using the dataframes, because for bones of the same day, they have different bone outline
# Not giving the source value, because we will apply the transformation to all the data without the data with source value 'GFP'
transformed_ss_bones = transform_bone_positions(aligned_ss_bone_outlines_smoothed, aligned_ss_bones, common_outline, x_num=x_num, y_num=y_num)
transformed_d5_bones = transform_bone_positions(aligned_d5_bone_outlines_smoothed, aligned_d5_bones, common_outline, x_num=x_num, y_num=y_num)
transformed_d10_bones = transform_bone_positions(aligned_d10_bone_outlines_smoothed, aligned_d10_bones, common_outline, x_num=x_num, y_num=y_num)
transformed_d15_bones = transform_bone_positions(aligned_d15_bone_outlines_smoothed, aligned_d15_bones, common_outline, x_num=x_num, y_num=y_num)
transformed_d30_bones = transform_bone_positions(aligned_d30_bone_outlines_smoothed, aligned_d30_bones, common_outline, x_num=x_num, y_num=y_num)

# Visualize the original and transformed bone positions for all the data sources
transformed_ss_bones_df = pd.concat(transformed_ss_bones)
transformed_d5_bones_df = pd.concat(transformed_d5_bones)
transformed_d10_bones_df = pd.concat(transformed_d10_bones)
transformed_d15_bones_df = pd.concat(transformed_d15_bones)
transformed_d30_bones_df = pd.concat(transformed_d30_bones)

# 10 mins


In [ ]:
# Save the transformed bone positions to CSV files
transformed_ss_bones_df.to_csv(f'{save_dir}/transformed_ss_bones_df.csv', index=False)
transformed_d5_bones_df.to_csv(f'{save_dir}/transformed_d5_bones_df.csv', index=False)
transformed_d10_bones_df.to_csv(f'{save_dir}/transformed_d10_bones_df.csv', index=False)
transformed_d15_bones_df.to_csv(f'{save_dir}/transformed_d15_bones_df.csv', index=False)
transformed_d30_bones_df.to_csv(f'{save_dir}/transformed_d30_bones_df.csv', index=False)

## 4. Spatial Probability Density Map Generation and Visualization

### 4.1 SPDM Generation (KDE Calculation)

In [ ]:
# Use the bone outline to generate the KDE for each source
#  Define the KDE function for each source, with weights based on z-aggregated points
def kde_for_source(df, bw_method='scott', bone_outline = None, binsize = 20):
    """
    Apply KDE to estimate the spatial distribution in the x-y plane for each source, 
    with weights aggregated for points with the same x, y but different z.

    Parameters:
    df : DataFrame containing 'Position.X', 'Position.Y', 'weights', and 'source'.
    bw_method : Bandwidth method for KDE (default is 'scott').
    bone_outline : The outline of the bone to limit the KDE calculation within the bone.
    binsize : The size of the bins for the grid used in the KDE calculation.

    Returns:
    kde_result : A dictionary containing the grid and KDE values for each source.
    """
    kde_results = {}
    sources = df['source'].unique()
    sources = sources[sources != 'GFP'] # Exclude the bone for the KDE calculation

    for source in sources:
        # Filter data for the current source
        source_data = df[df['source'] == source]

        # Group by Position.X and Position.Y, summing weights (or using counts as weights if no weights are given)
        if 'weights' in source_data.columns:
            source_data_agg = source_data.groupby(['Position.X', 'Position.Y'])['weights'].sum().reset_index()
        else:
            # If no weights are provided, use the count of occurrences as weights
            source_data_agg = source_data.groupby(['Position.X', 'Position.Y']).size().reset_index(name='weights')

        # Get the x and y values and aggregated weights
        x_vals = source_data_agg['Position.X']
        y_vals = source_data_agg['Position.Y']
        weights = source_data_agg['weights']
        if bone_outline is None:
            x_min, x_max = x_vals.min(), x_vals.max()
            y_min, y_max = y_vals.min(), y_vals.max()
        else:
            x_min, y_min = bone_outline.min(axis=0)
            x_max, y_max = bone_outline.max(axis=0)

        xi, yi = np.linspace(x_min, x_max, int((x_max - x_min)/binsize)+1), np.linspace(y_min, y_max, int((y_max - y_min)/binsize)+1)
        xi, yi = np.meshgrid(xi, yi)
        grid_points = np.vstack([xi.flatten(), yi.flatten()])
        common_grid = (xi, yi, grid_points)

        # Stack the x and y data for KDE input
        xy = np.vstack([x_vals, y_vals])

        # Perform the KDE with aggregated weights
        kde = gaussian_kde(xy, weights=weights, bw_method=bw_method)
        kde_values = kde(grid_points).reshape(xi.shape)

        # Store the results for each source
        kde_results[source] = kde_values

    return kde_results, common_grid




In [ ]:
# Instead of using the common bone outline (outmost),  we use the overlap of the bone outlines for the KDE calculation
kde_results_ss_overlap_bones, common_grid_overlap_bones = kde_for_source(transformed_ss_bones_df, binsize=20, bone_outline=common_outline)
kde_results_d5_overlap_bones, _ = kde_for_source(transformed_d5_bones_df, binsize=20, bone_outline=common_outline)
kde_results_d10_overlap_bones, _ = kde_for_source(transformed_d10_bones_df, binsize=20, bone_outline=common_outline)
kde_results_d15_overlap_bones, _ = kde_for_source(transformed_d15_bones_df, binsize=20, bone_outline=common_outline)
kde_results_d30_overlap_bones, _ = kde_for_source(transformed_d30_bones_df, binsize=20, bone_outline=common_outline)

In [ ]:

with open(f'{save_dir}/kde_results_ss_overlap_bones.pkl', 'wb') as f:
    pickle.dump(kde_results_ss_overlap_bones, f)
with open(f'{save_dir}/common_grid_overlap_bones.pkl', 'wb') as f:
    pickle.dump(common_grid_overlap_bones, f)
    
with open(f'{save_dir}/kde_results_d5_overlap_bones.pkl', 'wb') as f:
    pickle.dump(kde_results_d5_overlap_bones, f)
with open(f'{save_dir}/kde_results_d10_overlap_bones.pkl', 'wb') as f:
    pickle.dump(kde_results_d10_overlap_bones, f)
with open(f'{save_dir}/kde_results_d15_overlap_bones.pkl', 'wb') as f:
    pickle.dump(kde_results_d15_overlap_bones, f)
with open(f'{save_dir}/kde_results_d30_overlap_bones.pkl', 'wb') as f:
    pickle.dump(kde_results_d30_overlap_bones, f)


### 4.2 SPDM Visualization

In [ ]:

# Define function to create mask from outline (2D)
# TODO: do we need to change this to 3D?
def points_in_polygon(x_points, y_points, outline):
    path = Path(outline)
    points = np.vstack((x_points, y_points)).T
    return path.contains_points(points)



In [ ]:
# Visualize the KDE with the bone outline and optionally with the data points
def plot_kde_results_with_df(df, kde_results, common_grid, save_dir, color_map, bone_outline, filename=None, mesh = True, scatter = False):
    raw_sources = df['source'].unique()
    raw_sources = raw_sources[raw_sources != 'GFP']
    
    # The source should align with the color_maps keys order

    # Ensure that the sources are aligned with the keys of mk_color_map
    # First, get the color map keys, excluding 'Bone'
    color_map_keys = [key for key in color_map.keys() if key != 'GFP']

    # Reorder sources to match the order of color_map_keys
    sources = [source for source in color_map_keys if source in raw_sources]

    

    fig, axs = plt.subplots(len(sources), 1, figsize=(15 , 5 *len(sources)), sharex=True, sharey=True, constrained_layout=True)
    if len(sources) == 1:
        axs = [axs]  # since we only have one row, make it iterable

    for i, source in enumerate(sources):
        ax = axs[i]
        source_positions = df[df['source'] == source]
        
        # Plot the x, y of each bone group as the background
        if scatter:
            ax.scatter(source_positions['Position.X'], source_positions['Position.Y'], s=1, c=color_map[source], alpha=1)
        
        xi, yi, _ = common_grid # TODO: use the dict to save the common grid so we can use string to index
        zi = kde_results[source]
        # Masked the values outside the bone outline
        mask = points_in_polygon(xi.flatten(), yi.flatten(), bone_outline).reshape(xi.shape)
        zi[~mask] = 0

        # Filter only the non-zero values of zi for percentile calculation (i.e., exclude outside the mask)
        zi_inside_mask = zi[mask]

        # Normalize the values of zi for the full plot
        norm = Normalize(vmin=zi_inside_mask.min(), vmax=zi_inside_mask.max())
        normed_z = norm(zi)
        normed_z[~mask] = 0
        # Set the alpha transparency to 1 for test only
        # normed_z[mask] = 1    
        colors = np.array(plt.cm.colors.hex2color(color_map[source]))
        rgba_colors = np.zeros((*zi.shape, 4))
        rgba_colors[..., :3] = colors[:3]  # RGB values
        rgba_colors[..., -1] = normed_z  # Alpha transparency 
        
        # rgba_colors[zi < np.percentile(zi, 80)] = 0
        
        ax.set_title(source)
        xlim_min, ylim_min = bone_outline.min(axis=0)
        xlim_max, ylim_max = bone_outline.max(axis=0)
        # Convert them into integers
        xlim_min, xlim_max = int(xlim_min)-500, int(xlim_max)+500
        ylim_min, ylim_max = int(ylim_min)-300, int(ylim_max)+300
        
        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_xlabel('Position.X')
        ax.set_ylabel('Position.Y')
        if mesh:
            ax.pcolormesh(xi, yi, rgba_colors, shading='auto', rasterized=True)
        
        percentiles = np.arange(0, 81, 20)
        contour_levels = np.unique(np.percentile(norm(zi_inside_mask), percentiles))
        colors = color_map[source]
        if len(contour_levels) > 1:
            # print(contour_levels)
            # With changing linewidths, from the largest to the smallest
            # linewidths = np.linspace(0.5, 10, len(contour_levels))
            contour = ax.contour(xi, yi, normed_z, levels=contour_levels, linewidths=1, colors='black', alpha=0.5)
            

        # fmt = {level: f'{perc}%' for level, perc in zip(contour_levels, percentiles)}
        # ax.clabel(contour, contour_levels, inline=True, fmt=fmt, fontsize=8)

        ax.plot(bone_outline[:,0], bone_outline[:, 1], color='black')
        # TODO: optional
        # Find the peak in the kde result
        max_idx = np.unravel_index(np.argmax(zi), zi.shape)  # Index of maximum value in zi
        peak_x, peak_y = xi[max_idx], yi[max_idx]  # Get the coordinates of the peak

        # Plot the peak as a marker
        ax.plot(peak_x, peak_y, 's', markersize=5, label='Peak', color = 'black')  # Black square marker showing the global peak

    # Save the figure for the current bone group in the corresponding directory
    fig.suptitle(f'Spatial Distribution Estimation of {filename}', fontsize=16)
    if save_dir:
        if filename:
            fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{filename}.pdf')
            # fig_filename_pdf = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{filename}.pdf')
        else:
            fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{sources}.pdf') 
            # fig_filename_pdf = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{sources}.pdf')
        # Save it as pdf file
        fig.savefig(fig_filename, dpi=300, bbox_inches='tight')
        # Increase the maximum image size limit
        # Image.MAX_IMAGE_PIXELS = None
        
        # image = Image.open(fig_filename)
        # image.convert('RGB').save(fig_filename_pdf)
    #plt.close(fig)
    plt.show()

In [ ]:
# Visualize the KDE of MKs and Adipos with the bone outline
def plot_kde_results_MK_Adipos(kde_results, common_grid, save_dir, color_map, bone_outline, filename=None, mesh = True):
    sources = ['MKs','Adipos']

    fig, ax = plt.subplots(1, 1, figsize=(15 , 5))
    legend_patches = []
    for i, source in enumerate(sources):
        # if source == 'Adipos':
        #     continue
        # source_positions = df[df['source'] == source]
        # Plot the x, y of each bone group as the background
        # ax.scatter(source_positions['Position.X'], source_positions['Position.Y'], s=3, c=color_map[source], alpha=0.3)
        
        xi, yi, _ = common_grid # TODO: use the dict to save the common grid so we can use string to index
        zi = kde_results[source]
        # Masked the values outside the bone outline
        mask = points_in_polygon(xi.flatten(), yi.flatten(), bone_outline).reshape(xi.shape)
        zi[~mask] = 0
        
        # Make sure the values sum to 1
        # zi = zi / zi.sum()
        
        # Filter only the non-zero values of zi for percentile calculation (i.e., exclude outside the mask)
        zi_inside_mask = zi[mask]

        # Normalize the values of zi for the full plot
        norm = Normalize(vmin=zi_inside_mask.min(), vmax=zi_inside_mask.max())
        normed_z = norm(zi)
        normed_z[~mask] = 0
        
        # Set all no-zores values to 1
        # zi[zi > np.percentile(zi, 80)] = 1

        legend_patches.append(Patch(color=color_map[source], label=source, alpha = 0.5))

        colors = np.array(plt.cm.colors.hex2color(color_map[source]))
        rgba_colors = np.zeros((*zi.shape, 4))
        rgba_colors[..., :3] = colors[:3]  # RGB values
        
        # set the transparency into 5 levels based on normed_z
        # TODO
        # alpha_levels = np.linspace(0.1, 0.5, 5)
        # for i, alpha in enumerate(alpha_levels):
        #     rgba_colors[normed_z > np.percentile(zi_inside_mask, 20*i)] = [*colors[:3], alpha]
        rgba_colors[..., -1] = normed_z  # Alpha transparency
        # rgba_colors[normed_z < np.percentile(normed_z, 80)] = 0
        

        if mesh:
            ax.pcolormesh(xi, yi, rgba_colors, shading='auto', rasterized=True)

        
        percentiles = np.arange(0, 81, 20) # for 5 levels and if we want 10 levels we need to use np.arange(10, 101, 10)
        contour_levels = np.percentile(norm(zi_inside_mask), percentiles)
        # contour_levels = np.unique(np.percentile(norm(zi), percentiles))
        # Print the calculated contour levels
        print("Calculated Contour Levels:", contour_levels)
        rgba_colors[normed_z < contour_levels[-1]] = 0
        colors = color_map[source]
        if len(contour_levels) > 1:
            # With changing linewidths, from the largest to the smallest
            # linewidths = np.linspace(1, 5, len(contour_levels))
            # contour = ax.contour(xi, yi, normed_z, levels=contour_levels, linewidths=linewidths, colors=colors, alpha=0.5)

            # Only show the last two levels
            # linewidths = np.linspace(1, 5, len(contour_levels))
            linewidths = [2,4]
            contour = ax.contour(xi, yi, normed_z, levels=contour_levels[-2:], linewidths=linewidths, colors=colors, alpha=0.5)
            
            # fmt = {level: f'{100- perc}%' for level, perc in zip(contour_levels[-2:], percentiles[-2:])}
            # ax.clabel(contour, contour_levels[-2:], inline=True, fmt=fmt, fontsize=8, colors=colors)

    xlim_min, ylim_min = bone_outline.min(axis=0)
    xlim_max, ylim_max = bone_outline.max(axis=0)
    # Convert them into integers
    xlim_min, xlim_max = int(xlim_min)-500, int(xlim_max)+500
    ylim_min, ylim_max = int(ylim_min)-300, int(ylim_max)+300
    ax.set_title(filename)
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_xlabel('Position.X')
    ax.set_ylabel('Position.Y')
    ax.plot(bone_outline[:,0], bone_outline[:, 1], color='black')
    ax.legend(handles = legend_patches, loc='upper right')
    # Save the figure for the current bone group in the corresponding directory
    fig.suptitle(f'Spatial Probability Density Map', fontsize=16)
    if save_dir:
        if filename:
            fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{filename}.pdf')
            fig_filename_pdf = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{filename}.pdf')
        else:
            fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{sources}.pdf') 
            fig_filename_pdf = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_KDE_{sources}.pdf')
        fig.savefig(fig_filename, dpi=300)
        # Increase the maximum image size limit
        # Image.MAX_IMAGE_PIXELS = None
        
        # image = Image.open(fig_filename)
        # image.convert('RGB').save(fig_filename_pdf)
    #plt.close(fig)
    plt.show()


In [ ]:
def plot_1D_kde(kde_results, common_grid, save_dir, color_map, bone_outline, sources=['MKs', 'Adipos'], filename=None):

    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    
    for source in sources:
        xi, yi, _ = common_grid  # Get the common grid (xi for X, yi for Y)
        zi = kde_results[source]  # Get the KDE results for the current source
        
        # Mask values outside the bone outline
        mask = points_in_polygon(xi.flatten(), yi.flatten(), bone_outline).reshape(xi.shape)
        zi[~mask] = 0
        
        # Normalize the KDE results to ensure they sum to 1
        zi = zi / zi.sum()
        
        # Sum along the y-axis (rows) to project the KDE along the x-axis
        summed_zi_along_y = zi.sum(axis=0)  # Summing along rows gives us the KDE projection along X
        # Smooth the summed_zi_along_y with Gaussian filter
        smoothed_summed_zi = gaussian_filter1d(summed_zi_along_y, sigma=10)  # Adjust sigma as needed for smoothing
    
        # Plot the summed values as a 1D curve

        name_label = source

        # ax.plot(xi[0, :], summed_zi_along_y, label=f'{name_label} (original)', color=color_map[source], linewidth=2)
    
        # Plot the smoothed values as a dashed line in black
        ax.plot(xi[0, :], smoothed_summed_zi, label=f'{name_label}', color=color_map[source], linewidth=2) # linestyle='--', 
    # Set plot title, labels, and legend
    if filename:
        name_end = filename.split('_')[-1]
        ax.set_title(f'Marginal Distribution of {','.join(sources)} {name_end}', fontsize=16)
    else:
        ax.set_title('Marginal Distribution of MKs and Adipos', fontsize=16)
    ax.set_xlabel('Position.X', fontsize=14)
    ax.set_ylabel('Marginized Probability Density', fontsize=14)
    ax.legend(loc='upper right')
    # Set the min-max for y-axis
    # Get the current y-axis limits
    y_min, y_max = ax.get_ylim()

    # Set the y-ticks only at the min and max
    ax.set_yticks([y_min, y_max])

    # Set the y-tick labels to display min and max values
    ax.set_yticklabels(['min', 'max'])
    
    # Optionally, save the figure
    if save_dir:
        if filename:
            fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_1D_KDE_min_max_{filename}.pdf')
        else:
            fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_1D_KDE_MKs_Adipos.pdf')
        fig.savefig(fig_filename, dpi=300)
    
    plt.show()


In [ ]:

# Plot the KDE results per bone with the corresponding outline
plot_kde_results_with_df(transformed_ss_bones_df, kde_results_ss_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'd0')
plot_kde_results_with_df(transformed_d5_bones_df, kde_results_d5_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'd5')
plot_kde_results_with_df(transformed_d10_bones_df, kde_results_d10_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'd10')
plot_kde_results_with_df(transformed_d15_bones_df, kde_results_d15_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'd15')
plot_kde_results_with_df(transformed_d30_bones_df, kde_results_d30_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'd30')

In [ ]:
# Plot the KDE results per bone with the corresponding outline
plot_kde_results_with_df(transformed_ss_bones_df, kde_results_ss_overlap_bones, common_grid_overlap_bones, save_dir, hscs_color_map, common_outline, 'hsc_rd_d0', scatter=True)
plot_kde_results_with_df(transformed_d5_bones_df, kde_results_d5_overlap_bones, common_grid_overlap_bones, save_dir, hscs_color_map, common_outline, 'hsc_rd_d5', scatter=True)
plot_kde_results_with_df(transformed_d10_bones_df, kde_results_d10_overlap_bones, common_grid_overlap_bones, save_dir, hscs_color_map, common_outline, 'hsc_rd_d10', scatter=True)
plot_kde_results_with_df(transformed_d15_bones_df, kde_results_d15_overlap_bones, common_grid_overlap_bones, save_dir, hscs_color_map, common_outline, 'hsc_rd_d15', scatter=True)
plot_kde_results_with_df(transformed_d30_bones_df, kde_results_d30_overlap_bones, common_grid_overlap_bones, save_dir, hscs_color_map, common_outline, 'hsc_rd_d30', scatter=True)

In [ ]:
plot_1D_kde(kde_results_ss_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, filename='mk_adipos_d0')
plot_1D_kde(kde_results_d5_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, filename='mk_adipos_d5')
plot_1D_kde(kde_results_d10_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, filename='mk_adipos_d10')
plot_1D_kde(kde_results_d15_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, filename='mk_adipos_d15')
plot_1D_kde(kde_results_d30_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, filename='mk_adipos_d30')

In [ ]:
sources = ['MKs', 'Adipos', 'Sinusoids', 'MSCs', 'Neurons', 'Arteries']
plot_1D_kde(kde_results_ss_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, sources = sources, filename='all_d0')
plot_1D_kde(kde_results_d5_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, sources = sources, filename='all_d5')
plot_1D_kde(kde_results_d10_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, sources = sources, filename='all_d10')
plot_1D_kde(kde_results_d15_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, sources = sources, filename='all_d15')
plot_1D_kde(kde_results_d30_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, sources = sources, filename='all_d30')

In [ ]:
plot_kde_results_MK_Adipos(kde_results_ss_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'mk_adipos_d0')
plot_kde_results_MK_Adipos(kde_results_d5_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'mk_adipos_d5')
plot_kde_results_MK_Adipos(kde_results_d10_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'mk_adipos_d10')
plot_kde_results_MK_Adipos(kde_results_d15_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'mk_adipos_d15')
plot_kde_results_MK_Adipos(kde_results_d30_overlap_bones, common_grid_overlap_bones, save_dir, mk_color_map, common_outline, 'mk_adipos_d30')

## 5. Hierarchical Clustering and Heatmap

### 5.1 Hierarchical Clustering

In [ ]:

def calculate_kde_features(conditions, density_features, interested_source, common_grid, bone_outline):
    """
    Calculate KDE features for HSCs or RDs using nearest interpolation.

    Parameters:
    - conditions: List of tuples, where each tuple contains (condition_name, dataframe, kde_results_feature).
    - density_features: List of feature names (e.g., ['MKs', 'Adipos', 'Sinusoids', ...]).
    - interested_source: The source to calculate KDE for ('HSCs' or 'RDs').
    - common_grid: Tuple containing xi, yi, and other grid parameters.
    - bone_outline: Outline for masking KDE values outside the bone area.

    Returns:
    - DataFrame with KDE values for the specified source across all conditions.
    """
    xi, yi, _ = common_grid
    xi_flattened = xi.flatten()
    yi_flattened = yi.flatten()

    # Generate the bone mask once since it's the same for all conditions
    mask = points_in_polygon(xi_flattened, yi_flattened, bone_outline).reshape(xi.shape)

    # Prepare a list to collect DataFrames for each condition
    kde_feature_list = []

    # Loop over each condition and process KDE
    for condition_name, df, kde_results_feature in conditions:
        
        # Filter the data based on the interested source (HSCs or RDs)
        positions = df[df['source'] == interested_source][['Position.X', 'Position.Y']].values
        
        # Prepare a dictionary to store KDE features for the current condition
        kde_values_dict = {'Position.X': positions[:, 0], 'Position.Y': positions[:, 1], 'condition': condition_name}
        
        # Loop over each density feature and interpolate KDE values
        for source in density_features:
            zi = kde_results_feature[source].copy()
            zi[~mask] = 0  # Mask KDE values outside the bone outline
            
            # Normalize the KDE values within the mask to the range 0-1
            zi_inside_mask = zi[mask]
            norm = Normalize(vmin=zi_inside_mask.min(), vmax=zi_inside_mask.max())
            normed_z = norm(zi)
            normed_z[~mask] = 0
            
            # Interpolate KDE values for the interested source positions
            kde_values = griddata(
                (xi_flattened, yi_flattened),
                normed_z.flatten(),
                (positions[:, 0], positions[:, 1]),
                method='nearest',
                fill_value=0
            )
            
            # Add the interpolated KDE values to the dictionary
            kde_values_dict[source] = kde_values
        
        # Append the dictionary as a DataFrame for the current condition
        kde_feature_list.append(pd.DataFrame(kde_values_dict))
    
    # Concatenate all condition DataFrames into a single DataFrame
    return pd.concat(kde_feature_list, ignore_index=True)



In [ ]:
conditions_pos_kde = [
    ("d0", transformed_ss_bones_df, kde_results_ss_overlap_bones),
    ("d5", transformed_d5_bones_df, kde_results_d5_overlap_bones),
    ("d10", transformed_d10_bones_df, kde_results_d10_overlap_bones),
    ("d15", transformed_d15_bones_df, kde_results_d15_overlap_bones),
    ("d30", transformed_d30_bones_df, kde_results_d30_overlap_bones)
]

density_features = ['MKs', 'Adipos', 'Sinusoids', 'MSCs', 'Neurons', 'Arteries']

kde_values_for_hscs_nearest = calculate_kde_features(
    conditions=conditions_pos_kde,
    density_features=density_features,
    interested_source="HSCs",
    common_grid=common_grid_overlap_bones,
    bone_outline=common_outline
)
kde_values_for_rds_nearest = calculate_kde_features(
    conditions=conditions_pos_kde,
    density_features=density_features,
    interested_source="RDs",
    common_grid=common_grid_overlap_bones,
    bone_outline=common_outline
)


### 5.2 Heatmap Generation

In [ ]:
# Perform hierarchical clustering on the KDE values for HSCs
# Select the optimal number of clusters based on the Silhouette Score
hscs_features = kde_values_for_hscs_nearest[density_features]

# Perform hierarchical clustering using the 'ward' method
linkage_matrix_hscs = linkage(hscs_features, method='ward')

# Range of cluster numbers to evaluate
cluster_numbers = range(2,20)  # You can adjust this range based on your hscs_features
silhouette_scores = []

# Calculate the Silhouette Score for each number of clusters
for n_clusters in cluster_numbers:
    # Assign clusters based on the number of clusters (cutting the dendrogram)
    cluster_labels = fcluster(linkage_matrix_hscs, t=n_clusters, criterion='maxclust')
    
    # Calculate the Silhouette Score for the current number of clusters
    score = silhouette_score(hscs_features, cluster_labels)
    silhouette_scores.append(score)

# Find the optimal number of clusters based on the highest Silhouette Score
optimal_clusters = cluster_numbers[np.argmax(silhouette_scores)]
print(f'Optimal number of clusters: {optimal_clusters} with Silhouette Score: {max(silhouette_scores):.3f}')


plt.figure(figsize=(8, 6))
plt.plot(cluster_numbers, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Different Cluster Numbers')
plt.grid(True)
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_HSCs_silhouette.pdf')
plt.savefig(fig_filename, dpi=300)
plt.show()

In [ ]:
# Plot the hierarchical clustering heatmap of HSCs based on KDE values
# Assign clusters based on the optimal number of clusters (8 in this case)
kde_values_for_hscs_nearest['cluster'] = fcluster(linkage_matrix_hscs, t=optimal_clusters, criterion='maxclust')

# Add a color mapping for the conditions
condition_colors = {
    'd0': '#FB9A99',  # Pink
    'd5': '#CAB2D6',  # Lavender
    'd10': '#FF7F00', # - Orange
    'd15': '#A6CEE3', # - Light Blue
    'd30': '#33A02C' # - Green
}
row_colors_condition = kde_values_for_hscs_nearest['condition'].map(condition_colors)

# Define custom colors for each cluster
cluster_colors = {
    1: '#B15928', # - Brown
    2: '#6A3D9A', # - Purple
    3: '#1F78B4'   # Blue
}

# Map these colors to the 'cluster' column in your DataFrame
row_colors_cluster = kde_values_for_hscs_nearest['cluster'].map(cluster_colors)

# Combine row colors into a DataFrame
row_colors_df = pd.DataFrame({
    'Condition': row_colors_condition,  # Existing condition color mapping
    'Cluster': row_colors_cluster       # Custom cluster color mapping
})


# Create a larger figure to accommodate both clustermap and legend
fig = plt.figure(figsize=(12, 12))

min_value = hscs_features.values.min()
max_value = hscs_features.values.max()
# Create the clustermap with the row colors
g = sns.clustermap(
    hscs_features,
    row_linkage=linkage_matrix_hscs,
    col_cluster=False,  # Do not cluster the columns
    cmap='coolwarm',
    method = 'ward',
    figsize=(9, 12),
    row_colors=row_colors_df,  # Add row colors for conditions and clusters
    # standard_scale= 1,
    xticklabels=True,
    yticklabels=False,  # Hide row indices
    cbar_pos=(1.2, 0.7, 0.07, 0.18),  # Adjust x position of the color bar
    dendrogram_ratio=0.1, colors_ratio=0.02,
    vmin=min_value,  # Set the minimum color bar value
    vmax=max_value   # Set the maximum color bar value to the max of the data
)

# Set the color bar ticks and labels
colorbar = g.cax  # Access the color bar
colorbar.set_yticks([min_value, 0.50, max_value])  # Set color bar ticks at min and max values
colorbar.set_yticklabels([f'{min_value:.2f}',f'{0.50:.2f}', f'{max_value:.2f}'])  # Annotate color bar with min and max


# Add legends for the row colors (condition and cluster)
legend_handles = [
    mpatches.Patch(color=color, label=label) for label, color in condition_colors.items()
] + [
    mpatches.Patch(color=color, label=f'Cluster {i+1}') for i, color in enumerate(cluster_colors.values())
]

plt.legend(
    handles=legend_handles,
    title='Condition and Cluster',
    bbox_to_anchor=(1.2, -1),  # Align with color bar x position
)


    
# Show the plot
plt.suptitle('Hierarchical Clustering Heatmap of HSCs Based on KDE Values of Different Populations', fontsize=16, y=1)
save_dir = f'results/{datetime.today().strftime("%y%m%d")}_YL_PositionKDE'
if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_HSCs_heatmaps.pdf')
g.savefig(fig_filename, dpi=300)
plt.show()

In [ ]:
# Plot the hierarchical clustering heatmap of HSCs based on KDE values

rds_features = kde_values_for_rds_nearest[density_features]

# Perform hierarchical clustering using the 'ward' method
linkage_matrix_rds = linkage(rds_features, method='ward')

# Assign clusters based on the optimal number of clusters (8 in this case)
kde_values_for_rds_nearest['cluster'] = fcluster(linkage_matrix_rds, t=optimal_clusters, criterion='maxclust')

row_colors_condition = kde_values_for_rds_nearest['condition'].map(condition_colors)

# Map these colors to the 'cluster' column in your DataFrame
row_colors_cluster = kde_values_for_rds_nearest['cluster'].map(cluster_colors)

# Combine row colors into a DataFrame
row_colors_df = pd.DataFrame({
    'Condition': row_colors_condition,  # Existing condition color mapping
    'Cluster': row_colors_cluster       # Custom cluster color mapping
})


# Create a larger figure to accommodate both clustermap and legend
fig = plt.figure(figsize=(12, 12))

min_value = rds_features.values.min()
max_value = rds_features.values.max()
# Create the clustermap with the row colors
g = sns.clustermap(
    rds_features,
    row_linkage=linkage_matrix_rds,
    col_cluster=False,  # Do not cluster the columns
    cmap='coolwarm',
    method = 'ward',
    figsize=(9, 12),
    row_colors=row_colors_df,  # Add row colors for conditions and clusters
    # standard_scale= 1,
    xticklabels=True,
    yticklabels=False,  # Hide row indices
    cbar_pos=(1.2, 0.7, 0.07, 0.18),  # Adjust x position of the color bar
    dendrogram_ratio=0.1, colors_ratio=0.02,
    vmin=min_value,  # Set the minimum color bar value
    vmax=max_value   # Set the maximum color bar value to the max of the data
)

# Set the color bar ticks and labels
colorbar = g.cax  # Access the color bar
colorbar.set_yticks([min_value, 0.50, max_value])  # Set color bar ticks at min and max values
colorbar.set_yticklabels([f'{min_value:.2f}',f'{0.50:.2f}', f'{max_value:.2f}'])  # Annotate color bar with min and max

# Add legends for the row colors (condition and cluster)
legend_handles = [
    mpatches.Patch(color=color, label=label) for label, color in condition_colors.items()
] + [
    mpatches.Patch(color=color, label=f'Cluster {i+1}') for i, color in enumerate(cluster_colors.values())
]

plt.legend(
    handles=legend_handles,
    title='Condition and Cluster',
    bbox_to_anchor=(1.2, -1),  # Align with color bar x position
)


    
# Show the plot
plt.suptitle('Hierarchical Clustering Heatmap of RDs Based on KDE Values of Different Populations', fontsize=16, y=1)
save_dir = f'results/{datetime.today().strftime("%y%m%d")}_YL_PositionKDE'
if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_rd_heatmaps.pdf')
g.savefig(fig_filename, dpi=300)
plt.show()

In [ ]:
# Concatenate HSCs and RDs DataFrames
combined_kde_values = pd.concat([kde_values_for_hscs_nearest, kde_values_for_rds_nearest], ignore_index=True)

# Add a 'group' column to distinguish HSCs and RDs
combined_kde_values['group'] = combined_kde_values.apply(lambda x: 'HSCs' if x.name < len(kde_values_for_hscs_nearest) else 'RDs', axis=1)

# Extract KDE feature columns for both HSCs and RDs
hscs_features = kde_values_for_hscs_nearest[density_features]
rds_features = kde_values_for_rds_nearest[density_features]

# Combine HSCs and RDs KDE features into a single DataFrame
combined_features = pd.concat([hscs_features, rds_features], ignore_index=True)

# Perform hierarchical clustering on the combined features using the 'ward' method
linkage_matrix_combined = linkage(combined_features, method='ward')

# Range of cluster numbers to evaluate
cluster_numbers = range(2,11)  # You can adjust this range based on your hscs_features
silhouette_scores = []

# Calculate the Silhouette Score for each number of clusters
for n_clusters in cluster_numbers:
    # Assign clusters baon the number of clusters (cutting the dendrogram)
    cluster_labels = fcluster(linkage_matrix_combined, t=n_clusters, criterion='maxclust')
    
    # Calculate the Silhouette Score for the current number of clusters
    score = silhouette_score(combined_features, cluster_labels)
    silhouette_scores.append(score)

# Find the optimal number of clusters based on the highest Silhouette Score
optimal_clusters_combined = cluster_numbers[np.argmax(silhouette_scores)]
print(f'Optimal number of clusters: {optimal_clusters_combined} with Silhouette Score: {max(silhouette_scores):.3f}')


plt.figure(figsize=(8, 6))
plt.plot(cluster_numbers, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Different Cluster Numbers')
plt.grid(True)
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_silhouette_combined.pdf')
plt.savefig(fig_filename, dpi=300)
plt.show()

In [ ]:
# Define custom colors for each cluster
combined_kde_values['cluster'] = fcluster(linkage_matrix_combined, t=optimal_clusters_combined, criterion='maxclust')
row_colors_combined_cluster = combined_kde_values['cluster'].map(cluster_colors)
row_colors_combined_condition = combined_kde_values['condition'].map(condition_colors)
row_colors_combined_group = combined_kde_values['group'].map({'HSCs': 'gray', 'RDs': 'black'})

# Combine row colors into a DataFrame with condition, cluster, and group colors
row_colors_df_combined = pd.DataFrame({
    'Condition': row_colors_combined_condition,
    'Cluster': combined_kde_values['cluster'].map(cluster_colors),
    'Group': row_colors_combined_group
})

# Calculate the minimum and maximum value across all feature columns
min_value = combined_features.values.min()
max_value = combined_features.values.max()

# Plot the clustermap with custom color bar limits
g = sns.clustermap(
    combined_features,
    row_linkage=linkage_matrix_combined,
    col_cluster=False,
    cmap='coolwarm',
    method='ward',
    figsize=(9, 12),
    row_colors=row_colors_df_combined,
    xticklabels=True,
    yticklabels=False,
    cbar_pos=(1.2, 0.7, 0.07, 0.18),
    dendrogram_ratio=0.1,
    colors_ratio=0.02,
    vmin=min_value,  # Set the minimum color bar value
    vmax=max_value   # Set the maximum color bar value to the max of the data
)

# Set the color bar ticks and labels
colorbar = g.cax  # Access the color bar
colorbar.set_yticks([min_value, 0.50, max_value])  # Set color bar ticks at min and max values
colorbar.set_yticklabels([f'{min_value:.2f}',f'{0.50:.2f}', f'{max_value:.2f}'])  # Annotate color bar with min and max

# Add legends for conditions, clusters, and groups
legend_handles = [
    mpatches.Patch(color=color, label=label) for label, color in condition_colors.items()
] + [
    mpatches.Patch(color=color, label=f'Cluster {i+1}') for i, color in enumerate(cluster_colors.values())
] + [
    mpatches.Patch(color='gray', label='HSCs'), mpatches.Patch(color='black', label='RDs')
]

plt.legend(
    handles=legend_handles,
    title='Condition, Cluster, and Group',
    bbox_to_anchor=(1.2, -1)
)

# Show the plot with a title
plt.suptitle('Hierarchical Clustering Heatmap of HSCs and RDs Based on KDE Values', fontsize=16, y=1)
save_dir = f'results/{datetime.today().strftime("%y%m%d")}_YL_PositionKDE'
if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_HSCs_RDs_heatmaps.pdf')
g.savefig(fig_filename, dpi=300)
plt.show()

In [ ]:
# Update the clusters of the combined KDE values based on the optimal number of clusters
combined_kde_values['cluster'] = fcluster(linkage_matrix_combined, t=optimal_clusters, criterion='maxclust')


In [ ]:
filtered_combined_kde_values = combined_kde_values[combined_kde_values['group'] == 'HSCs']
# Define the desired order for the x-axis categories
desired_order = ['d0', 'd5', 'd10', 'd15', 'd30']

# Group by condition and cluster to count the number of HSCs in each cluster per condition
cluster_counts = filtered_combined_kde_values.groupby(['condition', 'cluster']).size().unstack(fill_value=0)
# Reindex the DataFrame to ensure the x-axis follows the specified order
cluster_counts = cluster_counts.reindex(desired_order)

# Extract custom colors for each cluster in the order of clusters
color_list = [cluster_colors[cluster] for cluster in sorted(cluster_colors.keys())]

# Plot the stacked bar chart with custom colors
cluster_counts.plot(kind='bar', stacked=True, color=color_list, figsize=(12, 8))

# Labeling the plot
plt.title('Cluster Composition by Condition', fontsize=16)
plt.xlabel('Condition')
plt.ylabel('Number of HSCs')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_HSCs_barplot_counts.pdf')
plt.savefig(fig_filename, dpi=300)

# Show the plot
plt.show()

In [ ]:
# Convert counts to percentages
cluster_percentages = cluster_counts.div(cluster_counts.sum(axis=1), axis=0)

# Plot the stacked bar chart with increased bar width
ax = cluster_percentages.plot(kind='bar', stacked=True, color=color_list, figsize=(12, 8), width=1)

# Number of conditions (categories) on the x-axis
num_conditions = len(cluster_percentages)
# Labeling the plot
ax.set_title('Cluster Composition by Condition (Percentage)', fontsize=16)
ax.set_xlabel('Condition')
ax.set_ylabel('Percentage of HSCs')
ax.set_ylim([0, 1])
ax.set_xlim([-0.5, num_conditions - 0.5]) 


ax.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_HSCs_bar_percentage.pdf')
plt.savefig(fig_filename, dpi=300)

# Show the plot
plt.show()

In [ ]:
filtered_combined_kde_values = combined_kde_values[combined_kde_values['group'] == 'RDs']
# Group by condition and cluster to count the number of HSCs in each cluster per condition
cluster_counts = filtered_combined_kde_values.groupby(['condition', 'cluster']).size().unstack(fill_value=0)
cluster_counts = cluster_counts.reindex(desired_order)
# Extract custom colors for each cluster in the order of clusters
color_list = [cluster_colors[cluster] for cluster in sorted(cluster_colors.keys())]

# Plot the stacked bar chart with custom colors
cluster_counts.plot(kind='bar', stacked=True, color=color_list, figsize=(12, 8))

# Labeling the plot
plt.title('Cluster Composition by Condition', fontsize=16)
plt.xlabel('Condition')
plt.ylabel('Number of HSCs')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_RDs_barplot_counts.pdf')
plt.savefig(fig_filename, dpi=300)

# Show the plot
plt.show()

In [ ]:
# Convert counts to percentages
cluster_percentages = cluster_counts.div(cluster_counts.sum(axis=1), axis=0)

# Plot the stacked bar chart with increased bar width
ax = cluster_percentages.plot(kind='bar', stacked=True, color=color_list, figsize=(12, 8), width=1)

# Number of conditions (categories) on the x-axis
num_conditions = len(cluster_percentages)
# Labeling the plot
ax.set_title('Cluster Composition by Condition (Percentage)', fontsize=16)
ax.set_xlabel('Condition')
ax.set_ylabel('Percentage of RDs')
ax.set_ylim([0, 1])
ax.set_xlim([-0.5, num_conditions - 0.5]) 


ax.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
fig_filename = os.path.join(save_dir, f'{datetime.today().strftime("%y%m%d")}_YL_RDs_bar_percentage.pdf')
plt.savefig(fig_filename, dpi=300)

# Show the plot
plt.show()